Задание 1
Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

In [6]:
import pandas as pd

movies = pd.read_csv("dataset/movies.csv")
ratings = pd.read_csv("dataset/ratings.csv")

ratings5 = ratings[ratings.rating == 5]

ratings5.groupby(['movieId'])['rating'].count()

movieId
1         47
2          7
3          6
5          3
6         24
          ..
184245     1
185135     1
187593     4
187595     1
187717     1
Name: rating, Length: 2954, dtype: int64

Задание 2
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [7]:
power = pd.read_csv("dataset/power.csv")

filtered_power = power.query('country == ["Latvia", "Lithuania", "Estonia"] and category == [4, 12, 21] and year >= 2005 and year <= 2010 and quantity > 0')

filtered_power['quantity'].sum()

240580.0

Задание 3
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni

In [8]:
tables = pd.read_html("https://www.finanz.ru/valyuty/v-realnom-vremeni", encoding='UTF-8')

len(tables) # 3

tables[0]

,0,1,2,3,4,5,6,7,8,9,10,11
0,ММВБ,2 829,"-0,6%",Nasdaq,11 864,"-0,1%",Биткойн,11 449,"0,2%",USD/RUB,771601,"0,1%"
1,РТС,1 156,"-0,6%",S&P500,3 512,"-0,6%",Нефть,423,"-0,4%",EUR/RUB,906220,"0,1%"
2,Dow,28 680,"-0,6%",FTSE100,5 970,"-0,5%",Золото,1 896,"0,3%",EUR/USD,11744,"0,0%"


Задание 4
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [11]:
movies = pd.read_csv("dataset/movies.csv")
ratings = pd.read_csv("dataset/ratings.csv")
al = ratings.merge(movies, on=["movieId"], how='inner')
f = al.pivot_table(index=['title','movieId'], values = 'rating', aggfunc='mean')
f.head()

f=f.reset_index()


def classification(i):
    if i  <= 2:
        return 'низкий рейтинг'
    elif i  <= 4 and i >2:
        return 'срдний рейтинг'
    elif i  >4.5:
        return 'высокий рейтинг'
    

f['class'] = f['rating'].apply(classification)
f.head()

,title,movieId,rating,class
0,'71 (2014),117867,4.0,срдний рейтинг
1,'Hellboy': The Seeds of Creation (2004),97757,4.0,срдний рейтинг
2,'Round Midnight (1986),26564,3.5,срдний рейтинг
3,'Salem's Lot (2004),27751,5.0,высокий рейтинг
4,'Til There Was You (1997),779,4.0,срдний рейтинг


Задание 5
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [15]:
import pandas as pd

keywords = pd.read_csv('keywords.csv')
keywords.head()

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

temp = {}

for region in geo_data:
    
    spisok_gorodov = geo_data[region]
    
    temp.update({key:value for key,value in zip(spisok_gorodov, [region for i in spisok_gorodov])})

keywords['region'] = keywords['keyword'].map(temp)
keywords['region'] = keywords['region'].fillna('undefined')
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


Задание 6
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [14]:
movies = pd.read_csv("dataset/movies.csv")
ratings = pd.read_csv("dataset/ratings.csv")
al = ratings.merge(movies, on=["movieId"], how='inner')
al.head()


years = []
for year in range(1950,2010):
    years.append(year)
    
number = len(years)
j = []
for i in range(1,number+1):
    j.append(i)
print (j)

def Years (title):
    for year in years:
        if str(year) in title:
            return year

    return 1900

al['year'] = al['title'].apply(Years)
al.head()

f = al.pivot_table(index=['year'], values = 'rating', aggfunc='mean')
f_sorted = f.sort_values(by = ['rating'], ascending = False)
f_sorted.head()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]


,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
